# Работа с данными

PyKEEN предоставляет большое количество встроенных датасетов в `pykeen.datasets`

In [4]:
from pykeen.datasets import Nations
dataset = Nations()

Основной класс для организации данных - `TripleFactory`. Сущности и отношения обычно хранятся в виде строк. Наша цель - получить векторные представления для этих строк. Процедура подготовки данных состоит из 2 шагов:

1. строки заменяются на числа (id)
2. из матрицы эмбеддингов получается эмбеддинг для нужной сущности

`TripleFactory` поддерживает набор уникальных меток для сущностей и отношений и строит маппинг строка => уникальный ID. Эти маппинги доступны через атрибуты `entity_id_to_label` и `relation_to_id`

## Создание датасета из 2 TSV файлов

Если данные представлены в виде 2 TSV файлов (train и test), состоящих из 3 столбцов, и мы уверены, что в тестовой части нет сущностей и отношений, которых не было в обучающей выборки, то модель можно обучить следующим образом:

In [21]:
from pykeen.pipeline import pipeline

result = pipeline(training='./data/train.txt', testing='./data/test.txt',
                  model='TransE',
                  dataset_kwargs={'create_inverse_triples': False})

Training epochs on cuda: 100%|██████████| 5/5 [00:00<00:00,  7.30epoch/s, loss=0.0061, prev_loss=0.00593]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.
Evaluating on cuda: 100%|██████████| 201/201 [00:00<00:00, 10.6ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.02s seconds


In [22]:
result.training.entity_id_to_label

{0: 'brazil',
 1: 'burma',
 2: 'china',
 3: 'cuba',
 4: 'egypt',
 5: 'india',
 6: 'indonesia',
 7: 'israel',
 8: 'jordan',
 9: 'netherlands',
 10: 'poland',
 11: 'uk',
 12: 'usa',
 13: 'ussr'}

Можно создать объекты класса `TriplesFactory` и работать с ними. В таком случае важно не забыть при создании тестовой фабрики указать параметр `entity_to_id` и `relation_to_id`. В противном случае ID из обучающей и тестовой фабрики не будут соответсвовать, что приведет к некорретным результатам.

In [29]:
from pykeen.triples import TriplesFactory

training = TriplesFactory.from_path('./data/train.txt', create_inverse_triples=False)
testing = TriplesFactory.from_path('./data/test.txt', 
                                   entity_to_id=training.entity_to_id,
                                   relation_to_id=training.relation_to_id,
                                   create_inverse_triples=False)

result = pipeline(training=training, testing=testing,
                  model='TransE')                                   

Training epochs on cuda: 100%|██████████| 5/5 [00:00<00:00,  7.79epoch/s, loss=0.0061, prev_loss=0.00627]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.
Evaluating on cuda: 100%|██████████| 201/201 [00:00<00:00, 10.6ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.02s seconds


Если датасет не разбит на 2 части, его можно разбить при помощи метода `TriplesFactory.split`. 

In [30]:
tf = TriplesFactory.from_path('./data/train.txt', create_inverse_triples=False)
training, testing = tf.split([.9, .1])

result = pipeline(training=training, testing=testing,
                  model='TransE')                                   

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [1365, 160]
Training epochs on cuda: 100%|██████████| 5/5 [00:00<00:00,  8.48epoch/s, loss=0.00574, prev_loss=0.00584]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=160.
Evaluating on cuda: 100%|██████████| 160/160 [00:00<00:00, 7.27ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.03s seconds
